# Charging Cost Calculator Documentation
This notebook documents the design and methodology of the Charging Cost Calculator. It is not meant for production use.
to run the calculator with user input data and real rate data, use chargingCostCalculator.py


This calculator models electricity costs for charging electric vehicles (or any general electricity use) based on two user inputs:
* Hourly average energy use (in kWh)
* Hourly peak energy use (which is the highest kW power draw across 4 15-minute intervals within that hour)

The calculator provides simulated electrical costs for any or all rates tracked by OpenEI's Electricity Rate Database: https://apps.openei.org/USURDB/

## Dependencies
The following libraries are required to run this analysis

In [4]:
import numpy as np
import pickle

# internal libraries
from lib.rate_parsing import rateProcess

## Data and Inputs
Two Primary Inputs:
* rate data (parsed from URDB JSON database extract)
* energy and power use data (provided as user input)

### Rate Data
This script uses test case rates to validate it is working properly which are input here. Test case rates are defined in test_data.py and are read in from a pickle file here. test_data.py must be run before those rates are available for use. Test cases are dictionaries formatted the same as the parsed JSON files from URDB. In the main tool, this parsing is done in a preprocessing step so that all rate data is available for sequential calculations.

Rate data is stored as a dictionary where each calculation input is stored as a separate key-value pair.

### Energy and Power Data
For demonstration and testing purposes we provide simple synthetic inputs that is that defined in test_data.py and are read in from a pickle file here. test_data.py must be run before the test power and energy inputs are available.

In the main tool this is read in from an excel workbook. Users will be able to specify either a monthly or
single load curve. However, the tool will always use a monthly load curve. single load curve will be repeated for each month.

In [5]:
# load in test rates data and test energy and power curve from 
# testing/test_data.pkl must run test_rates.py first to generate this file
with open('testing/test_data.pkl', 'rb') as f:
    tcTier = pickle.load(f)
    tcTOU = pickle.load(f)
    tcNRG = pickle.load(f)
    tcPower = pickle.load(f)

f.close()

# process rates into analysis ready format using rate process function
# rateProcess functions are in lib/rate_parsing.py
tcTier = rateProcess(tcTier)
tcTOU = rateProcess(tcTOU)


# Daily Charging Parameters
What days of the week that vehicles charge affects modeled energy bills becuase charging on weekends is cheaper in many cases (where there are TOU rates).

Users select how many days a week they charge their vehicles. At least one charging day is assumed to be a weekend day to account for likely priority for at least one weekend charging day in any given week. E.g., a vehicle that needs charging 5 days a week to cover normal weekday operations would likely have its Friday charge delayed until Saturday when costs are cheaper (though this assumes managed or at least scheduled charging). We ignore leap years.

ex:
* 4 charge days = 3 weekdays, 1 weekend
* 5 charge days = 4 weekdays, 1 weekend
* 6 charge days = 5 weekdays, 1 weekends

The number of weekdays and weekends charged in each month are modeled as:
* weekdays = (n...5)/7 * n days per month
* weekends = (n...2)/7 (*) n days per month

note that this creates fractional days

In [6]:

# there are only 7 possible outputs (14 lists) from this so this may change to a
# hardcoded lookup table in the future

def daysMonth(chargeDays):
    """ 
    Returns a nested list of weekdays/weekends in each month for a given 
    number of charge days (rounded to 2 sig digits)
    """
    if chargeDays > 7:
        raise ValueError('Charge days cannot be greater than 7')

    days = [31,28,31,30,31,30,31,31,30,31,30,31]
    weekdays = 5 if chargeDays > 5 else chargeDays - 1
    weekends = 2 if chargeDays > 6 else 1

    return [[np.round(days[i] * weekdays / 7, 2) for i in range(12)], # refactor
            [np.round(days[i] * weekends / 7, 2) for i in range(12)]]


In [18]:
# test function and return results for use below.
days = 5
daylist = daysMonth(days)

# ------------------------------------------------------------------------------

# for easier testing of the calculator, we simplify days per month. This input
# is used in the rest of this notebook
daylist = [[8,8,8,8,8,8,8,8,8,8,8,8],
           [2,2,2,2,2,2,2,2,2,2,2,2]]

### Energy/Power preprocessing functions

These values are repeatedly used so we precalculate them.

* Summed Energy - monthly energy use based on the number of days charged per month. This energy use value is used for calculating tiered energy charges and to create per-kWh energy use.

* Max Power - Max power per month used for flat demand charge calculation

In [8]:

def nrgUse(nrg, daysMonth):
    """
    returns a list of monthly energy use in kWh based on a energy use input and 
    daysMonth list
    """
    use = [sum(i) for i in nrg]

    # add daysMonth[0] (weekday) and daysMonth[1] (weekend) to get total days
    # in month
    nDays = np.nansum(daysMonth, axis=0).tolist()

    # pairwise multiply nrgUse and nDays to get total monthly energy use 
    # for each month
    return [use[x] * nDays[x] for x in range(12)]


def getMaxPower(power):
    # was more complicated but is now a single line function
        return [np.max(power) for i in range(12)]



In [9]:

# run to test functions and create variables for later use
useNRG = nrgUse(tcNRG, daylist)
maxPower = getMaxPower(tcPower)

print(useNRG)
print(maxPower)


[600, 600, 600, 600, 600, 600, 600, 600, 600, 600, 600, 600]
[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4]


# Calculate Tiered Energy Rate Costs
The functions in this section are all used for tiered rate data prep and calculation. Tiered energy rates base the cost of energy depending on how much energy is consumed per month.

Control flow of the main script determines if a given rate is a tiered rate (opposed to TOU) and directs calculation to the operations in this function.

## Tiered Energy Cost Calculator
This function calculates the cost of energy for a tiered rate in a given month using 
* the total energy amount in kWh
* rates corresponding to each tier
* costs of energy in each tier

The calculation method assumes that tiered rates are implemented on a marginal basis. (i.e. only the portion of energy that falls within a tier's min-max range is assessed at that rate). 

For example:
A user that uses 100 kWh per month with a 10kWh tier max for tier 0 and a 50kWh max for tier 1, with per kwh costs of $0.10, $0.15 and $0.20 in tiers 0,1,2 would be charged:

0.10*(10-0) + 0.15*(50-10) + 0.20*(100-50) = 1 + 6 + 10 = 17 ($17)

*Note: while this is a common way of implementing tiered rates, not all tiered rates work this way. However, we do not have enough information to apply different calculations on a per rate basis.*

In [10]:

def tierCalc(unit, costTier, maxVal):
    """
    Calculates cost for a given tier structure AND 
    energy/power use. 

    this function is used in the nrgTierCalc function below to calculate cost of 
    tiered energy rates and is also used for tiered flat demand charges.

    unit: energy or power use
    costTier: tiered rate structure
    maxVal: max value for each tier

    output: list of cost for each month
    """

    # generate containers
    unitVal = unit
    cost = 0

    # loop through costTier to calculate cost
    for i in range(len(costTier)):

        # define upper and lower values
        lower = 0 if i == 0 else maxVal[i-1]
        upper = maxVal[i] if i < len(maxVal) else unit + 1
        
        # if eng is within tier margin calculate cost on unitVal and tier
        # and break loop
        if unit > lower and unit <= upper:
            cost += (unitVal * costTier[i])
            break
        
        # otherwise add in full tier cost and subtract amount of energy
        # accounted for from unitVal and continue loop
        elif unit > upper:
            band = upper - lower
            cost += (band * costTier[i])
            unitVal -= band

    return cost

def nrgTierCalc(nrg, tierRate, tierMax):
    """
    simple list comprehension to calculate energy cost for each month based
    on tiered rate structure and energy use using the tierCalc function
    """
    return [tierCalc(nrg[i], tierRate[i], tierMax[i]) for i in range(12)]

            


### Calculating expected output value for Tiered Energy Cost

* 600kWh of electricity per month
* period 0 (months 0-5)
    * cost (3,4,5)
    * max (10,20)
* period 1 (months 6-11)
    * cost (2,3,4)
    * max (20,30) 

Period 0 cost = 3*(10-0) + 4*(20-10) + 5*(600 - 20)  
Period 1 cost  = 2*(20-0) + 3*(30-20) + 4*(600 - 30)

output should be list of 12 split evenly between p0 and p1

In [19]:
p0 = 3*(10-0) + 4*(20-10) + 5*(600 - 20)
p1 = 2*(20-0) + 3*(30-20) + 4*(600 - 30)

expectedVal = [p0, p0, p0, p0, p0, p0, p1, p1, p1, p1, p1, p1]

### Run and Test Tiered Energy Cost Calculator

In [20]:

# set values based on tiered rate structure test data
rateVals = tcTier['nrgTierRates']
maxVals = tcTier['nrgTierMax']

# run function (save output for later use)
tieredNRGcost = nrgTierCalc(useNRG, rateVals, maxVals)

# test results
print(tieredNRGcost)
print(expectedVal)
if tieredNRGcost == expectedVal:
    print('calculator output equal to expected value, test passed')
else:
    print('calculator output not equal to expected value, test failed')

[2970, 2970, 2970, 2970, 2970, 2970, 2350, 2350, 2350, 2350, 2350, 2350]
[2970, 2970, 2970, 2970, 2970, 2970, 2350, 2350, 2350, 2350, 2350, 2350]
calculator output equal to expected value, test passed


# Calculate TOU Energy Costs
TOU energy rates base the cost of energy depending on when (by hour, weekday/weekend, and month) the energy is consumed. 

Control flow of the main script determines if a given rate is a TOU rate (opposed to tiered) and directs calculation to the operations in this section.

To calculate TOU cost the script:
* Calculates the Hadamard Product (pairwise matrix multiplication) of the 12X24 matrices of TOU rates and energy use
* Sum costs into single day (weekend and weekday) energy costs
* multiply those weekend and weekday energy costs by the number of days for each within the month (as determined by operating days)

In [13]:
def nrgTOUCalc(nrg, touWeekday, touWeekend, daysInMonth):
    """
    Calculates energy cost for a given TOU structure using a
    monthly energy use input.

    expects input of 12 x 2 x 24 array for touSchedule and 12 x 24 array for nrg
    """

    touSchedule = [touWeekday, touWeekend] # make array

    # set of array calculations to calculate energy cost
    costArr = np.multiply(nrg, touSchedule)
    costArr = np.nansum(costArr, axis=2)

    # multiply by days in month (weekend and weekday)
    costArr = np.multiply(costArr, daysInMonth)

    # add weekend and weekday costs
    costArr = np.nansum(costArr, axis=0).tolist()
    
    return costArr


# Testing

### Calculating expected output value for Tiered Energy Cost

* weekend and weekday rates alternate months with these 24 hour rate periods:   
  [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4],  
  [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6],  
* daily energy use for weekdays and weekends are:
  [1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4]
* 10 days per month (8 weekday and 2 weekend)

ADD EXPLANATION

monthly energy costs will be:  
period 0 = ((1\*2\*6) + (2\*2\*6) + (3\*4\*6) + (4\*4\*6)) * 10  
period 1 = ((1\*2\*6) + (2\*2\*6) + (3\*6\*6) + (4\*6\*6)) * 10


In [21]:
p0 = ((1*2*6) + (2*2*6) + (3*4*6) + (4*4*6)) * 10
p1 = ((1*2*6) + (2*2*6) + (3*6*6) + (4*6*6)) * 10

expectedVal = [p0, p1, p0, p1, p0, p1, p0, p1, p0, p1, p0, p1]


In [22]:
# set inputs
touWeekday = tcTOU['nrgTOUWkdRates']
touWeekend = tcTOU['nrgTOUWkeRates']

# run function (save output for later use)
touNRGcost = nrgTOUCalc(tcNRG, touWeekday, touWeekend, daylist)

print(touNRGcost)
print(expectedVal)
if touNRGcost == expectedVal:
    print('calculator output equal to expected value, test passed')
else:
    print('calculator output not equal to expected value, test failed')

[2040, 2880, 2040, 2880, 2040, 2880, 2040, 2880, 2040, 2880, 2040, 2880]
[2040, 2880, 2040, 2880, 2040, 2880, 2040, 2880, 2040, 2880, 2040, 2880]
calculator output equal to expected value, test passed


# Calculate Flat Demand charges
This function calculates the cost of power for a flat (non-time-based) demand charge in a given month using 
* the max power amount in kW
* rates corresponding to each tier (or single rate)
* max value in each tier (or none)

The calculation method assumes that tiered kWh rates are implemented on a marginal basis. (i.e. only the portion of power that falls within a tier's min-max range is assessed at that rate). 

For example:
A user that uses 100 kW peak in a month with a 10kW tier max for tier 0 and a 50kW max for tier 1, with per kwh costs of $10, $15 and $20 in tiers 0,1,2 would be charged:  
10*(10-0) + 15*(50-10) + 20*(100-50) = 100 + 600 + 1000 = 1700 ($1,700)

Non-tiered rates are simply calculated as max kW times set rate. e.g. $10/kW * 100kW = $1,000

Flat demand charges might change seasonally, this script accomodates differnt demand charges for each month. *Note that this script does not support ratcheting demand charges.*

Control flow of the main script determines if a rate has a flat demand charge and then directs the script to this set of functions


In [26]:
def flatDemandCalc(rate, mx, maxPower):
    """
    Calculates flat demand charges for a given flat demand schedule
    using a monthly max power input.

    control flow is based on whether or not there is a tiered rate structure 
    in the rate input

    rate: flat demand rate structure
    mx: max values for each tier
    maxPower: max power for each month
    """
    # if max is false (empty) then there is a flat rate and the function 
    # calculates that 
    if mx == False:
        print('flat rate')
        # flatten rate[0] to remove nested lists
        xsublist = [x for y in rate for x in y]
        # return maxPower * rate for each month
        return (np.multiply(maxPower, xsublist)
                .tolist())
    
    # if rate[1][0] (max vals) is not empty, then there is a tiered rate and the
    # function calls the tierCalc function to calculate the demand charge for 
    # each month
    else:
        print('tiered rate')
        return [tierCalc(maxPower[i], rate[i], mx[i]) for i in range(12)]


# Testing

### Calculating expected output value for Flat Demand Charge
This function calculates both tiered and non-tiered charges

#### tiered
* max power 4kW
* period 0 (months 0-5)
    * cost (2,3,4)
    * max (1,2)
* period 1 (months 6-11)
    * cost (3,4,5)
    * max (2,3) 

Period 0 cost = 2*(1-0) + 3*(2-1) + 4*(4-2)  
Period 1 cost = 3*(2-0) + 4*(3-2) + 5*(4-3)

output should be list of 12 split evenly between p0 and p1

#### non-tiered
* max power 4kW
* period 0 (months 0-5)
    * cost (2)
* period 1 (months 6-11)
    * cost (3)

period 0 = 2\*4 = 8  
period 1 = 3\*4 = 12

output should be list of 12 split evenly between p0 and p1

In [27]:
# tiered flat demand expected value

p0 = 2*(1-0) + 3*(2-1) + 4*(4-2)
p1 = 3*(2-0) + 4*(3-2) + 5*(4-3)

expectedValTier = [p0, p0, p0, p0, p0, p0, p1, p1, p1, p1, p1, p1]


# untiered expected value

p0 = 2*4
p1 = 3*4 

expectedVal = [p0, p0, p0, p0, p0, p0, p1, p1, p1, p1, p1, p1]

In [28]:

# ----------------------test tiered rate (in tcTier) -------------------------
# set input
mx = tcTier['demandFlatMax']
rate = tcTier['demandFlatRates']

# run function (save output for later use)
flatDemandTier = flatDemandCalc(rate, mx, maxPower)

print(flatDemandTier)
print(expectedValTier)
if flatDemandTier == expectedValTier:
    print('calculator output equal to expected value, test passed\n')
else:
    print('calculator output not equal to expected value, test failed\n')


# ----------------------test flat rate (in tcTOU) -----------------------------
# set input
mx = tcTOU['demandFlatMax']
rate = tcTOU['demandFlatRates']

# run function (save output for later use)
FlatDemand = flatDemandCalc(rate, mx, maxPower)

print(FlatDemand)
print(expectedVal)
if FlatDemand == expectedVal:
    print('calculator output equal to expected value, test passed')
else:
    print('calculator output not equal to expected value, test failed')


tiered rate
[13, 13, 13, 13, 13, 13, 15, 15, 15, 15, 15, 15]
[13, 13, 13, 13, 13, 13, 15, 15, 15, 15, 15, 15]
calculator output equal to expected value, test passed

flat rate
[8, 8, 8, 8, 8, 8, 12, 12, 12, 12, 12, 12]
[8, 8, 8, 8, 8, 8, 12, 12, 12, 12, 12, 12]
calculator output equal to expected value, test passed


# Calculate TOU demand charge

This function calculates the cost of power for a TOU demand charge in each month
* hourly max power in kW
* rates corresponding to each hour

TOU demand charges are assessed against the highest power in each period of demand charges for each period in each month (note that some TOU demand charges charge by day rather than month, however the script does not support those rates)

For example: 
if the each day in the month is split between two periods with the first (am) having a charge of $10/kW and pm having a charge of $20/kW  
and the max monthly power use in am is 10kW and the max monthly power in pm is 20kW then the total TOU demand charge for the month would be:  
10\*10 + 20\*20 = 100 + 400 = 500 ($500)

 *Note that this script does not support ratcheting demand charges.*

Control flow of the main script determines if a rate has a flat demand charge and then directs the script to this set of functions

In [32]:
def demandTOUCalc(rate, power):
  '''
  takes in a 12 x 24 array for rate and a 12 x 24 array for power
  and calculates the demand charge for each month

  rate: demand rate structure
  power: hourly power for each month
  '''

  # get unique values in rate (these are the periods)
  rateU = [list(set(i)) for i in rate]
    
  # get max power for each period (complicated list comprehension to get
  # max power for each period across all 12 months)
  maxPower = [[max(power[z][i] for i in range(24) if rate[z][i] == x)
                  for x in rateU[z]] 
                    for z in range(len(rateU))]
                    
  return [sum(maxPower[n][i] * rateU[n][i] for i in range(len(rateU[n])))
             for n in range(12)]

## Testing

### Calculating expected output value for TOU Demand Charge

* power input is  
  [1, 1, 1, 1, 1, 1, 4, 4, 4, 4, 4, 4, 3, 3, 3, 3, 3, 3, 2, 2, 2, 2, 2, 2]
* rate input alternates between the following two lists  
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]  
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5]

period 0 = max(1, 1, 1, 1, 1, 1, 4, 4, 4, 4, 4, 4) * 0 + max(3, 3, 3, 3, 3, 3, 2, 2, 2, 2, 2, 2) * 3 = 0 + 9 = 9  
period 1 = max(1, 1, 1, 1, 1, 1, 4, 4, 4, 4, 4, 4) * 0 + max(3, 3, 3, 3, 3, 3, 2, 2, 2, 2, 2, 2) * 5 = 0 + 15 = 15   

output should be list of 12 alternating between p0 and p1

In [31]:
p0 = np.max([1, 1, 1, 1, 1, 1, 4, 4, 4, 4, 4, 4]) * 0 + np.max([3, 3, 3, 3, 3, 3, 2, 2, 2, 2, 2, 2]) * 3
p1 = np.max([1, 1, 1, 1, 1, 1, 4, 4, 4, 4, 4, 4]) * 0 + np.max([3, 3, 3, 3, 3, 3, 2, 2, 2, 2, 2, 2]) * 5

expectedVal = [p0, p1, p0, p1, p0, p1, p0, p1, p0, p1, p0, p1]


In [33]:
rate = tcTOU['demandTOUwkdRates']
power = tcPower

touDemand = demandTOUCalc(rate, power)

print(touDemand)
print(expectedVal)
if touDemand == expectedVal:
    print('calculator output equal to expected value, test passed')
else:
    print('calculator output not equal to expected value, test failed')


[9, 15, 9, 15, 9, 15, 9, 15, 9, 15, 9, 15]
[9, 15, 9, 15, 9, 15, 9, 15, 9, 15, 9, 15]
calculator output equal to expected value, test passed
